In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [4]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("Retry Config Done!")

Retry Config Done!


In [7]:
# ===============================
# SASSA SIDEKICK AGENT — TOOLS
# ===============================

from typing import Dict, Any, List


# 1. INTENT CLASSIFICATION
def classify_intent(user_query: str) -> Dict[str, Any]:
    """
    Classify the user's intent based on their query.

    Returns:
        {"status": "success", "data": {"intent": str}}
        or
        {"status": "error", "error_message": str}
    """
    try:
        query = user_query.lower()

        if any(w in query for w in ["child support", "csg", "child grant"]):
            intent = "child_support_grant"
        elif any(w in query for w in ["srd", "350", "covid grant"]):
            intent = "srd_grant"
        elif any(w in query for w in ["old age", "pension"]):
            intent = "old_age_grant"
        elif "uif" in query:
            intent = "uif_claim"
        elif any(w in query for w in ["home affairs", "id", "passport"]):
            intent = "home_affairs_document"
        else:
            intent = "general_social_service_query"

        return {"status": "success", "data": {"intent": intent}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 2. CONTEXT EXTRACTION
def extract_user_context(user_query: str) -> Dict[str, Any]:
    """
    Extracts context from the user query (age, employment status, etc.)

    Returns:
        {"status": "success", "data": {"context": dict}}
    """
    try:
        context = {}

        # Simple heuristic rules – you can expand using regex if needed
        if "unemployed" in user_query.lower():
            context["employment_status"] = "unemployed"
        if "working" in user_query.lower():
            context["employment_status"] = "employed"
        if "child" in user_query.lower():
            context["has_child"] = True

        return {"status": "success", "data": {"context": context}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 3. POLICY RETRIEVAL (RAG SIMULATION)
def retrieve_policy_info(query: str) -> Dict[str, Any]:
    """
    Retrieve SASSA policy information based on a query.
    In a real system, this performs RAG search over documents.

    Returns:
        {"status": "success", "data": {"policy": str}}
    """
    try:
        mock_db = {
            "child_support_grant": "Eligibility: Caregiver of a child under 18, means test required.",
            "old_age_grant": "Eligibility: 60+ years old, South African citizen or permanent resident.",
            "srd_grant": "Eligibility: Unemployed, no income, SA resident, age 18+.",
        }

        result = mock_db.get(query.lower(), "No policy information found.")

        return {"status": "success", "data": {"policy": result}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 4. ELIGIBILITY CHECK
def check_eligibility(context: Dict[str, Any], grant_type: str) -> Dict[str, Any]:
    """
    Check if the user is eligible for a specific grant.

    Returns:
        {"status": "success", "data": {"eligible": bool, "reason": str}}
    """
    try:
        # Simplified logic example:
        if grant_type == "child_support_grant":
            if not context.get("has_child"):
                return {"status": "success", "data": {"eligible": False, "reason": "You must be caring for a child."}}

        return {"status": "success", "data": {"eligible": True, "reason": "You meet the basic criteria."}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 5. REQUIRED DOCUMENT LIST
def list_required_documents(grant_type: str) -> Dict[str, Any]:
    """
    List all required documents for a given grant type.

    Returns:
        {"status": "success", "data": {"documents": list}}
    """
    try:
        docs = {
            "child_support_grant": [
                "Caregiver ID",
                "Child birth certificate",
                "Proof of income (if applicable)"
            ],
            "old_age_grant": [
                "ID Book / Smart ID",
                "Proof of income",
                "Bank statements"
            ],
            "srd_grant": [
                "ID Number",
                "Phone number registered with SASSA"
            ]
        }

        return {"status": "success", "data": {"documents": docs.get(grant_type, [])}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 6. MISSING DOCUMENT CHECK
def check_missing_documents(user_docs: List[str], required_docs: List[str]) -> Dict[str, Any]:
    """
    Compare user-provided documents with required documents.

    Returns:
        {"status": "success", "data": {"missing": list}}
    """
    try:
        missing = [doc for doc in required_docs if doc not in user_docs]

        return {"status": "success", "data": {"missing": missing}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 7. PROCESS STEPS
def generate_process_steps(grant_type: str, context: Dict[str, Any]) -> Dict[str, Any]:
    """
    Generate step-by-step instructions for applying for a grant.

    Returns:
        {"status": "success", "data": {"steps": list}}
    """
    try:
        steps = {
            "child_support_grant": [
                "Go to your nearest SASSA office.",
                "Bring the required documents.",
                "Complete the application form.",
                "Submit fingerprints and biometrics.",
                "Receive receipt with reference number."
            ],
            "old_age_grant": [
                "Visit nearest SASSA office.",
                "Take your ID and proof of income.",
                "Complete the Old Age Grant form.",
                "Submit supporting documents.",
                "Wait for approval SMS."
            ]
        }

        return {"status": "success", "data": {"steps": steps.get(grant_type, ["No steps available"])}}    

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 8. SIMULATED STATUS CHECK
def simulate_status_check(id_number: str) -> Dict[str, Any]:
    """
    Simulate a grant application status check for demonstration purposes.

    Returns:
        {"status": "success", "data": {"status": str}}
    """
    try:
        # Very basic—randomize or logic could be added
        if id_number.endswith("1"):
            status = "Approved"
        elif id_number.endswith("2"):
            status = "Pending"
        else:
            status = "Rejected"

        return {"status": "success", "data": {"status": status}}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


In [8]:
# SASSA Sidekick Agent with custom function tools
sassa_sidekick_agent = LlmAgent(
    name="sassa_sidekick_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are SASSA Sidekick — a smart, friendly assistant that helps South Africans
    understand SASSA processes, requirements, eligibility, and policy information.

    When responding to the user, follow these rules:

    1. Use the correct tool based on intent:
        • classify_intent — classify what the user wants  
        • extract_user_context — extract details such as ID number, grant type, dates  
        • retrieve_policy_info — retrieve SASSA policy rules  
        • check_eligibility — check if the user qualifies for a grant  
        • list_required_documents — list the documents needed  
        • check_missing_documents — compare provided vs required documents  
        • generate_process_steps — generate step-by-step guidance  
        • simulate_status_check — simulate a SASSA status lookup  

    2. Always check the tool’s "status" field.
       If status == "error":
           - Explain the issue clearly and give a next step.

    3. When the user asks a multi-step question:
       - Start with a simple summary
       - Then provide a structured breakdown (policy, requirements, steps, etc.)

    4. Keep responses clear, simple, and appropriate for everyday South Africans.
    """,
    tools=[
        classify_intent,
        extract_user_context,
        retrieve_policy_info,
        check_eligibility,
        list_required_documents,
        check_missing_documents,
        generate_process_steps,
        simulate_status_check,
    ],
)

print("✅ SASSA Sidekick Agent created with custom function tools")
print("🔧 Available tools:")
print("  • classify_intent - Determine what the user wants to do")
print("  • extract_user_context - Extract details from user messages")
print("  • retrieve_policy_info - Get SASSA rules & policy data")
print("  • check_eligibility - Evaluate if user qualifies for a grant")
print("  • list_required_documents - Show documents needed for a grant")
print("  • check_missing_documents - Compare user docs vs required docs")
print("  • generate_process_steps - Provide step-by-step instructions")
print("  • simulate_status_check - Simulate checking a grant application status")


✅ SASSA Sidekick Agent created with custom function tools
🔧 Available tools:
  • classify_intent - Determine what the user wants to do
  • extract_user_context - Extract details from user messages
  • retrieve_policy_info - Get SASSA rules & policy data
  • check_eligibility - Evaluate if user qualifies for a grant
  • list_required_documents - Show documents needed for a grant
  • check_missing_documents - Compare user docs vs required docs
  • generate_process_steps - Provide step-by-step instructions
  • simulate_status_check - Simulate checking a grant application status


In [9]:
# Test the SASSA Sidekick Agent
sassa_runner = InMemoryRunner(agent=sassa_sidekick_agent)

# Example end-to-end test prompt
_ = await sassa_runner.run_debug(
    "Hi, I want to check if I qualify for a Child Support Grant. I earn R3500 monthly and my child is 8 years old."
)


 ### Created new session: debug_session_id

User > Hi, I want to check if I qualify for a Child Support Grant. I earn R3500 monthly and my child is 8 years old.
